In [110]:
# Import libraries
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)
import requests
import matplotlib.pyplot as plt
import json
import re
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [111]:
# Read csv from raw github link and create dataframe
url = 'https://raw.githubusercontent.com/dianeooty/datascience_salary/main/Resources/layoffs.csv'
layoffs_df = pd.read_csv(url)

In [112]:
# Read csv from raw github link and create dataframe
url = 'https://raw.githubusercontent.com/dianeooty/datascience_salary/main/Resources/Levels_Fyi_Salary_Data.csv'
salaries_df = pd.read_csv(url)

In [113]:
# View dataframe info
salaries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                62642 non-null  object 
 1   company                  62637 non-null  object 
 2   level                    62523 non-null  object 
 3   title                    62642 non-null  object 
 4   totalyearlycompensation  62642 non-null  int64  
 5   location                 62642 non-null  object 
 6   yearsofexperience        62642 non-null  float64
 7   yearsatcompany           62642 non-null  float64
 8   tag                      61788 non-null  object 
 9   basesalary               62642 non-null  int64  
 10  stockgrantvalue          62642 non-null  float64
 11  bonus                    62642 non-null  float64
 12  gender                   43102 non-null  object 
 13  otherdetails             40137 non-null  object 
 14  cityid                

In [114]:
# Display dataframe
salaries_df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
0,6/7/2017 11:33,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000,...,0,0,0,0,0,0,0,0,NaN,NaN
1,6/10/2017 17:11,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,6/11/2017 14:53,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000,...,0,0,0,0,0,0,0,0,NaN,NaN
3,6/17/2017 0:23,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000,...,0,0,0,0,0,0,0,0,NaN,NaN
4,6/20/2017 10:58,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [115]:
layoffs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2545 entries, 0 to 2544
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              2545 non-null   object 
 1   location             2545 non-null   object 
 2   industry             2543 non-null   object 
 3   total_laid_off       1746 non-null   float64
 4   percentage_laid_off  1694 non-null   float64
 5   date                 2543 non-null   object 
 6   stage                2539 non-null   object 
 7   country              2545 non-null   object 
 8   funds_raised         2297 non-null   float64
dtypes: float64(3), object(6)
memory usage: 179.1+ KB


In [116]:
# Display dataframe
layoffs_df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,N26,Berlin,Finance,71.0,0.04,2023-04-28,Series E,United States,1700.0
1,Providoor,Melbourne,Food,NaN,1.00,2023-04-28,Unknown,Australia,NaN
2,Dropbox,SF Bay Area,Other,500.0,0.16,2023-04-27,Post-IPO,United States,1700.0
3,Vroom,New York City,Transportation,120.0,0.11,2023-04-27,Post-IPO,United States,1300.0
4,Greenhouse,New York City,Recruiting,100.0,0.12,2023-04-27,Private Equity,United States,110.0


In [117]:
# Convert timestamp format
salaries_df['timestamp'] = pd.to_datetime(salaries_df['timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')
salaries_df['timestamp'] = salaries_df['timestamp'].astype('datetime64[ns]')

In [118]:
# Convert timestamp column to dates only and add a new column
salaries_df['date'] = pd.to_datetime(salaries_df['timestamp']).dt.date
salaries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   timestamp                62642 non-null  datetime64[ns]
 1   company                  62637 non-null  object        
 2   level                    62523 non-null  object        
 3   title                    62642 non-null  object        
 4   totalyearlycompensation  62642 non-null  int64         
 5   location                 62642 non-null  object        
 6   yearsofexperience        62642 non-null  float64       
 7   yearsatcompany           62642 non-null  float64       
 8   tag                      61788 non-null  object        
 9   basesalary               62642 non-null  int64         
 10  stockgrantvalue          62642 non-null  float64       
 11  bonus                    62642 non-null  float64       
 12  gender                   43102 n

In [119]:
# Convert date column to datetime format
salaries_df['date'] = salaries_df['date'].astype('datetime64[ns]')
salaries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   timestamp                62642 non-null  datetime64[ns]
 1   company                  62637 non-null  object        
 2   level                    62523 non-null  object        
 3   title                    62642 non-null  object        
 4   totalyearlycompensation  62642 non-null  int64         
 5   location                 62642 non-null  object        
 6   yearsofexperience        62642 non-null  float64       
 7   yearsatcompany           62642 non-null  float64       
 8   tag                      61788 non-null  object        
 9   basesalary               62642 non-null  int64         
 10  stockgrantvalue          62642 non-null  float64       
 11  bonus                    62642 non-null  float64       
 12  gender                   43102 n

In [120]:
# Display dataframe to confirm changes
salaries_df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education,date
0,2017-06-07 11:33:00,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000,...,0,0,0,0,0,0,0,NaN,NaN,2017-06-07
1,2017-06-10 17:11:00,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0,...,0,0,0,0,0,0,0,NaN,NaN,2017-06-10
2,2017-06-11 14:53:00,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000,...,0,0,0,0,0,0,0,NaN,NaN,2017-06-11
3,2017-06-17 00:23:00,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000,...,0,0,0,0,0,0,0,NaN,NaN,2017-06-17
4,2017-06-20 10:58:00,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0,...,0,0,0,0,0,0,0,NaN,NaN,2017-06-20


In [11]:
# Install opencage for API calls
pip install opencage

In [12]:
# Import additional libraries for opencage
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [13]:
# Import key to access API
from key import oc_key

In [14]:
# Assign variable for API calls with key
geocoder = OpenCageGeocode(oc_key)

In [15]:
# Create a list of locations for API calls
location = []
[location.append(x) for x in salaries_df.location]
print(location)

In [16]:
# Use set to get distinct locations and cast to list for ordering
distinct = set(location)
distinct_location = list(distinct)

In [17]:
# Create empty lists to hold latitude and longitude values from each city
lng = []
lat = []

# Iterate through API results to extract latitude and longitude values and append to empty lists
for x in distinct_location:
    results = geocoder.geocode(x)
    lng.append(results[0]['geometry']['lng'])
    lat.append(results[0]['geometry']['lat'])

In [18]:
# Create dataframe for cities and coordinates
coord_df = pd.DataFrame(
            {
                "city": distinct_location,
                "latitude": lat,
                "longitude": lng
            }
)

# Display Dataframe
coord_df.head()

In [19]:
# View dataframe dimensions
coord_df.shape

In [20]:
# Check data types
coord_df.dtypes

In [21]:
# Save coord_df to csv
coord_df.to_csv('Resources/coordinates.csv', index=False)

In [121]:
# Display dataframe
salaries_df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education,date
0,2017-06-07 11:33:00,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000,...,0,0,0,0,0,0,0,NaN,NaN,2017-06-07
1,2017-06-10 17:11:00,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0,...,0,0,0,0,0,0,0,NaN,NaN,2017-06-10
2,2017-06-11 14:53:00,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000,...,0,0,0,0,0,0,0,NaN,NaN,2017-06-11
3,2017-06-17 00:23:00,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000,...,0,0,0,0,0,0,0,NaN,NaN,2017-06-17
4,2017-06-20 10:58:00,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0,...,0,0,0,0,0,0,0,NaN,NaN,2017-06-20


In [122]:
# View dataframe dimensions
salaries_df.shape

(62642, 30)

In [123]:
# Check data types
salaries_df.dtypes

timestamp                  datetime64[ns]
company                            object
level                              object
title                              object
totalyearlycompensation             int64
location                           object
yearsofexperience                 float64
yearsatcompany                    float64
tag                                object
basesalary                          int64
stockgrantvalue                   float64
bonus                             float64
gender                             object
otherdetails                       object
cityid                              int64
dmaid                             float64
rowNumber                           int64
Masters_Degree                      int64
Bachelors_Degree                    int64
Doctorate_Degree                    int64
Highschool                          int64
Some_College                        int64
Race_Asian                          int64
Race_White                        

In [124]:
# IF NEEDED: Import coordinates.csv to merge and save as coord_df
# Read csv from raw github link and create dataframe
url = 'https://raw.githubusercontent.com/dianeooty/datascience_salary/main/Resources/coordinates.csv'
coord_df = pd.read_csv(url)

In [125]:
# # Merge the dataframes using left join to add coordinates to salaries_df
new_df = pd.merge(salaries_df,coord_df,how='left',left_on='location',right_on='city')
new_df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education,date,city,latitude,longitude
0,2017-06-07 11:33:00,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000,...,0,0,0,0,NaN,NaN,2017-06-07,"Redwood City, CA",37.486324,-122.232523
1,2017-06-10 17:11:00,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0,...,0,0,0,0,NaN,NaN,2017-06-10,"San Francisco, CA",37.779026,-122.419906
2,2017-06-11 14:53:00,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000,...,0,0,0,0,NaN,NaN,2017-06-11,"Seattle, WA",47.603832,-122.330062
3,2017-06-17 00:23:00,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000,...,0,0,0,0,NaN,NaN,2017-06-17,"Sunnyvale, CA",37.368830,-122.036349
4,2017-06-20 10:58:00,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0,...,0,0,0,0,NaN,NaN,2017-06-20,"Mountain View, CA",37.389389,-122.083210


In [126]:
# View column names
new_df.columns

Index(['timestamp', 'company', 'level', 'title', 'totalyearlycompensation',
       'location', 'yearsofexperience', 'yearsatcompany', 'tag', 'basesalary',
       'stockgrantvalue', 'bonus', 'gender', 'otherdetails', 'cityid', 'dmaid',
       'rowNumber', 'Masters_Degree', 'Bachelors_Degree', 'Doctorate_Degree',
       'Highschool', 'Some_College', 'Race_Asian', 'Race_White',
       'Race_Two_Or_More', 'Race_Black', 'Race_Hispanic', 'Race', 'Education',
       'date', 'city', 'latitude', 'longitude'],
      dtype='object')

In [127]:
# Reorder columns
new_df = new_df[['timestamp','date', 'company', 'level', 'title', 'totalyearlycompensation',
       'location', 'latitude', 'longitude','yearsofexperience', 'yearsatcompany', 'tag', 'basesalary',
       'stockgrantvalue', 'bonus', 'gender', 'otherdetails', 'cityid', 'dmaid',
       'rowNumber', 'Masters_Degree', 'Bachelors_Degree', 'Doctorate_Degree',
       'Highschool', 'Some_College', 'Race_Asian', 'Race_White',
       'Race_Two_Or_More', 'Race_Black', 'Race_Hispanic', 'Race', 'Education']]

In [128]:
# Display dataframe
new_df.head()

,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
0,2017-06-07 11:33:00,2017-06-07,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,...,0,0,0,0,0,0,0,0,NaN,NaN
1,2017-06-10 17:11:00,2017-06-10,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,2017-06-11 14:53:00,2017-06-11,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,2017-06-17 00:23:00,2017-06-17,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,2017-06-20 10:58:00,2017-06-20,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [129]:
# View dataframe's info
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62642 entries, 0 to 62641
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   timestamp                62642 non-null  datetime64[ns]
 1   date                     62642 non-null  datetime64[ns]
 2   company                  62637 non-null  object        
 3   level                    62523 non-null  object        
 4   title                    62642 non-null  object        
 5   totalyearlycompensation  62642 non-null  int64         
 6   location                 62642 non-null  object        
 7   latitude                 62642 non-null  float64       
 8   longitude                62642 non-null  float64       
 9   yearsofexperience        62642 non-null  float64       
 10  yearsatcompany           62642 non-null  float64       
 11  tag                      61788 non-null  object        
 12  basesalary               62642 n

In [130]:
# Replace gender column null values with unknown
new_df['gender'].fillna("Unknown", inplace = True)

In [131]:
# Replace gender column incorrect values with unknown
new_df['gender'] = new_df['gender'].str.replace("Title: Senior Software Engineer","Unknown")

In [132]:
# Check values updated
new_df['gender'].value_counts()

Male       35702
Unknown    19541
Female      6999
Other        400
Name: gender, dtype: int64

In [133]:
# Replace null values with unknown
new_df['Race'].fillna("Unknown", inplace = True)

In [134]:
# Replace null values with unknown
new_df['Education'].fillna("Unknown", inplace = True)

In [135]:
# Drop unwanted columns
new_df = new_df.drop(columns=['tag', 'otherdetails', 'cityid', 'dmaid',
       'rowNumber', 'Masters_Degree', 'Bachelors_Degree', 'Doctorate_Degree',
       'Highschool', 'Some_College', 'Race_Asian', 'Race_White',
       'Race_Two_Or_More', 'Race_Black', 'Race_Hispanic'])

In [136]:
# Convert datatypes
new_df['basesalary'] = new_df['basesalary'].astype('int')
new_df['stockgrantvalue'] = new_df['stockgrantvalue'].astype('int')
new_df['bonus'] = new_df['bonus'].astype('int')

In [137]:
# Check dataframe's info
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62642 entries, 0 to 62641
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   timestamp                62642 non-null  datetime64[ns]
 1   date                     62642 non-null  datetime64[ns]
 2   company                  62637 non-null  object        
 3   level                    62523 non-null  object        
 4   title                    62642 non-null  object        
 5   totalyearlycompensation  62642 non-null  int64         
 6   location                 62642 non-null  object        
 7   latitude                 62642 non-null  float64       
 8   longitude                62642 non-null  float64       
 9   yearsofexperience        62642 non-null  float64       
 10  yearsatcompany           62642 non-null  float64       
 11  basesalary               62642 non-null  int32         
 12  stockgrantvalue          62642 n

In [138]:
# Save new_df to csv
new_df.to_csv('Resources/salaries_cleaned.csv')

In [139]:
# Display dataframe
new_df.head()

,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education
0,2017-06-07 11:33:00,2017-06-07,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown
1,2017-06-10 17:11:00,2017-06-10,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown
2,2017-06-11 14:53:00,2017-06-11,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown
3,2017-06-17 00:23:00,2017-06-17,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown
4,2017-06-20 10:58:00,2017-06-20,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown


In [140]:
# Display dataframe
layoffs_df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,N26,Berlin,Finance,71.0,0.04,2023-04-28,Series E,United States,1700.0
1,Providoor,Melbourne,Food,NaN,1.00,2023-04-28,Unknown,Australia,NaN
2,Dropbox,SF Bay Area,Other,500.0,0.16,2023-04-27,Post-IPO,United States,1700.0
3,Vroom,New York City,Transportation,120.0,0.11,2023-04-27,Post-IPO,United States,1300.0
4,Greenhouse,New York City,Recruiting,100.0,0.12,2023-04-27,Private Equity,United States,110.0


In [141]:
# Check dataframe's info
layoffs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2545 entries, 0 to 2544
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              2545 non-null   object 
 1   location             2545 non-null   object 
 2   industry             2543 non-null   object 
 3   total_laid_off       1746 non-null   float64
 4   percentage_laid_off  1694 non-null   float64
 5   date                 2543 non-null   object 
 6   stage                2539 non-null   object 
 7   country              2545 non-null   object 
 8   funds_raised         2297 non-null   float64
dtypes: float64(3), object(6)
memory usage: 179.1+ KB


In [142]:
# Replace null values with 0
layoffs_df['total_laid_off'].fillna(0, inplace = True)

In [143]:
# Convert datatypes
layoffs_df['date'] = layoffs_df['date'].astype('datetime64[ns]')

In [144]:
# Convert to percentage amount
layoffs_df['percentage_laid_off'] = layoffs_df['percentage_laid_off']*100

In [145]:
# Convert datatypes
layoffs_df['total_laid_off'] = layoffs_df['total_laid_off'].astype('int')

In [146]:
# Display dataframe
layoffs_df.head(30)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,N26,Berlin,Finance,71,4.0,2023-04-28,Series E,United States,1700.0
1,Providoor,Melbourne,Food,0,100.0,2023-04-28,Unknown,Australia,NaN
2,Dropbox,SF Bay Area,Other,500,16.0,2023-04-27,Post-IPO,United States,1700.0
3,Vroom,New York City,Transportation,120,11.0,2023-04-27,Post-IPO,United States,1300.0
4,Greenhouse,New York City,Recruiting,100,12.0,2023-04-27,Private Equity,United States,110.0
5,Rebellion Defense,Washington D.C.,Data,90,NaN,2023-04-27,Series B,United States,150.0
6,Poppulo,Denver,HR,85,NaN,2023-04-27,Acquired,United States,30.0
7,Megaport,Brisbane,Infrastructure,50,16.0,2023-04-27,Post-IPO,Australia,98.0
8,Airtasker,Sydney,Retail,45,20.0,2023-04-27,Series C,Australia,26.0
9,Chief,New York City,Other,43,14.0,2023-04-27,Series B,United States,140.0


In [147]:
# Check dataframe's info
layoffs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2545 entries, 0 to 2544
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   company              2545 non-null   object        
 1   location             2545 non-null   object        
 2   industry             2543 non-null   object        
 3   total_laid_off       2545 non-null   int32         
 4   percentage_laid_off  1694 non-null   float64       
 5   date                 2543 non-null   datetime64[ns]
 6   stage                2539 non-null   object        
 7   country              2545 non-null   object        
 8   funds_raised         2297 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(5)
memory usage: 169.1+ KB


In [148]:
# Drop unwanted columns
layoffs_df = layoffs_df.drop(columns=['funds_raised'])

In [149]:
# Save layoffs_df to csv
layoffs_df.to_csv('Resources/layoffs_cleaned.csv')